In [1]:
# you need to install paho client for this to work and you should have a running
# mqtt broker on your local system. 
#pip install paho-mqtt

In [2]:
# mqtt client
import paho.mqtt.client as paho
import syft as sy
import json
from syft.core.broadcast_message import SyftBroadcastMessageWithoutReply, SyftBroadcastMessageWithReply

In [3]:
def deserialize_msg_from_json(msg_payload):
    """
    this is dummy for this notebook only - future use protobuf
    it is also highly insecure.
    """
    msg = json.loads(msg_payload.decode('utf-8'))
    klass = msg['class']
    specs = sy.core.broadcast_message.BroadcastMessageSpecs(msg['specs'])
    syft_msg_obj = eval(klass)(specs = specs, msg_id = msg['msg_id'])
    return syft_msg_obj

In [4]:
class PahoClient():

    def __init__(self, host="localhost", port=1883):
        self.host = host
        self.port = port
        self.client = paho.Client()
        self.client.on_subscribe = self.on_subscribe
        self.client.on_message = self.on_message
        self.client.on_publish = self.on_publish
        self.client.on_log = self.on_log

    def on_log(self, client, obj, level, string):
        print(string)

    def on_publish(self, client, userdata, mid):
        pass
    
    def on_subscribe(self, client, userdata, mid, granted_qos):
        print("Subscribed: "+str(mid)+" "+str(granted_qos))

    def on_message(self, client, userdata, msg):
        print(msg.topic+" "+str(msg.qos)+" "+str(len(msg.payload)))        
        to_process = deserialize_msg_from_json(msg.payload)
        return self.processor(msg = to_process)

    def send(self, topic, msg):
        (rc, mid) = self.client.publish(topic, str(msg), qos=1, retain=False)

    def connect(self):
        self.client.connect(self.host, self.port)

    def listen(self, topic, processor, background = True):
        
        self.processor = processor
        
        self.client.subscribe(topic, qos=1)
        if not background:
            self.client.loop_forever()
        else:
            self.client.loop_start()


In [5]:
class BroadcastConnection(PahoClient, sy.core.io.connection.ClientConnection): 
    def send_msg_with_reply(self, topic, msg): 
        if not self.client.is_connected(): 
            self.connect() 
        self.send(topic, msg) 
    def send_msg_without_reply(self, topic, msg): 
        if not self.client.is_connected(): 
            self.connect() 
        self.send(topic, msg) 
              


In [6]:
conn = BroadcastConnection()

In [7]:
conn.send_msg_without_reply('test', 'hello mqtt!')   

Sending CONNECT (u0, p0, wr0, wq0, wf0, c1, k60) client_id=b''
Sending PUBLISH (d0, q1, r0, m1), 'b'test'', ... (11 bytes)


If you had a mosquito client listening in your system, you'd hear the message!

you can run the client like this, after you've installed it {insert installation link}
> mosquitto_sub -t "test"


now let's build a network and routes

In [8]:
vm = sy.VirtualMachine(name = "tf-cuda-vm")
device = sy.Device(name = "home-pc")
domain = sy.Domain(name = "Lisa")
network = sy.Network(name = "science-team")

In [9]:
domain.id

<UID:edab7a65-7895-4aa2-8cb4-cef43c45da75>

In [10]:
# we are going to make them all talk over the pub-sub route
broadcast_route_network_domain = sy.core.io.route.BroadcastRoute(topic = 'science-team', connection = conn)
broadcast_route_domain_device = sy.core.io.route.BroadcastRoute(topic = 'lisa-devices', connection = conn)
broadcast_route_device_vm = sy.core.io.route.BroadcastRoute(topic = 'home-pc', connection = conn)

In [11]:
network_client = network.get_client(routes=[broadcast_route_network_domain])

In [12]:
specs = sy.core.broadcast_message.BroadcastMessageSpecs(
    {
    'compute': 'heavy',
    'data': ['fitness', 'heartrate'],
    'model_type': 'neural-net',
    'framework-type': 'tensorflow'
    })
msg = SyftBroadcastMessageWithoutReply(specs = specs)
msg = json.dumps(msg.to_dict())

In [13]:
network_client.send_broadcast_msg_without_reply(msg = msg)

Sending CONNECT (u0, p0, wr0, wq0, wf0, c1, k60) client_id=b''
Sending PUBLISH (d0, q1, r0, m2), 'b'science-team'', ... (186 bytes)


let's run a domain client that can listen on messages coming from the network and can broadcast to devices

In [14]:
domain_client = domain.get_client(routes=[broadcast_route_network_domain, broadcast_route_domain_device])

In [15]:
domain_client.routes

[science-team, lisa-devices]

In [16]:
domain.listen_on_route(route = broadcast_route_network_domain)

Sending SUBSCRIBE (d0, m3) [(b'science-team', 1)]
Received CONNACK (0, 0)
Sending PUBLISH (d1, q1, r0, m1), 'b'test'', ... (11 bytes)
Received PUBACK (Mid: 2)
Received SUBACK
Subscribed: 3 (1,)
Received PUBACK (Mid: 1)


In [17]:
network_client.send_broadcast_msg_without_reply(msg = msg)

Sending PUBLISH (d0, q1, r0, m4), 'b'science-team'', ... (186 bytes)
Received PUBACK (Mid: 4)


In [18]:
network_client.routes

[science-team]

Received PUBLISH (d0, q1, r0, m1), 'science-team', ...  (186 bytes)
Sending PUBACK (Mid: 1)
science-team 1 186
got a broadcast message
